In [1]:
import xlnet
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import model_utils

W0716 17:06:20.779078 140390586017536 deprecation_wrapper.py:119] From /home/jupyter/xlnet/model_utils.py:293: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [2]:
import sentencepiece as spm
from prepro_utils import preprocess_text, encode_ids

sp_model = spm.SentencePieceProcessor()
sp_model.Load('sp10m.cased.v5.model')

True

In [3]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/subjectivity/subjectivity-negative-bm.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/subjectivity/subjectivity-positive-bm.txt

In [4]:
with open('subjectivity-negative-bm.txt','r') as fopen:
    texts = fopen.read().split('\n')
labels = [0] * len(texts)

with open('subjectivity-positive-bm.txt','r') as fopen:
    positive_texts = fopen.read().split('\n')
labels += [1] * len(positive_texts)
texts += positive_texts

assert len(labels) == len(texts)

In [5]:
from prepro_utils import preprocess_text, encode_ids

def tokenize_fn(text):
    text = preprocess_text(text, lower= True)
    return encode_ids(sp_model, text)

In [6]:
MAX_SEQ_LENGTH = 128

SEG_ID_A   = 0
SEG_ID_B   = 1
SEG_ID_CLS = 2
SEG_ID_SEP = 3
SEG_ID_PAD = 4

special_symbols = {
    "<unk>"  : 0,
    "<s>"    : 1,
    "</s>"   : 2,
    "<cls>"  : 3,
    "<sep>"  : 4,
    "<pad>"  : 5,
    "<mask>" : 6,
    "<eod>"  : 7,
    "<eop>"  : 8,
}

VOCAB_SIZE = 32000
UNK_ID = special_symbols["<unk>"]
CLS_ID = special_symbols["<cls>"]
SEP_ID = special_symbols["<sep>"]
MASK_ID = special_symbols["<mask>"]
EOD_ID = special_symbols["<eod>"]

input_ids, input_masks, segment_ids = [], [], []

for text in tqdm(texts):
    tokens_a = tokenize_fn(text)
    if len(tokens_a) > MAX_SEQ_LENGTH - 2:
        tokens_a = tokens_a[:(MAX_SEQ_LENGTH - 2)]
        
    tokens = []
    segment_id = []
    for token in tokens_a:
        tokens.append(token)
        segment_id.append(SEG_ID_A)
    tokens.append(SEP_ID)
    segment_id.append(SEG_ID_A)
    tokens.append(CLS_ID)
    segment_id.append(SEG_ID_CLS)
    
    input_id = tokens
    input_mask = [0] * len(input_id)
    if len(input_id) < MAX_SEQ_LENGTH:
        delta_len = MAX_SEQ_LENGTH - len(input_id)
        input_id = [0] * delta_len + input_id
        input_mask = [1] * delta_len + input_mask
        segment_id = [SEG_ID_PAD] * delta_len + segment_id
    
    input_ids.append(input_id)
    input_masks.append(input_mask)
    segment_ids.append(segment_id)

100%|██████████| 9962/9962 [00:01<00:00, 6276.27it/s]


In [7]:
kwargs = dict(
      is_training=True,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.0,
      dropatt=0.0,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clamp_len=-1)

xlnet_parameters = xlnet.RunConfig(**kwargs)
xlnet_config = xlnet.XLNetConfig(json_path='output-model4/config.json')

W0716 17:06:32.997750 140390586017536 deprecation_wrapper.py:119] From /home/jupyter/xlnet/xlnet.py:63: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.



In [8]:
epoch = 10
batch_size = 10
warmup_proportion = 0.1
num_train_steps = int(len(input_ids) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)
print(num_train_steps, num_warmup_steps)
learning_rate = 2e-5

training_parameters = dict(
      decay_method = 'poly',
      train_steps = num_train_steps,
      learning_rate = learning_rate,
      warmup_steps = num_warmup_steps,
      min_lr_ratio = 0.0,
      weight_decay = 0.00,
      adam_epsilon = 1e-8,
      num_core_per_host = 1,
      lr_layer_decay_rate = 1,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.0,
      dropatt=0.0,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clip = 1.0,
      clamp_len=-1,)

9962 996


In [9]:
class Parameter:
    def __init__(self, decay_method, warmup_steps, weight_decay, adam_epsilon, 
                num_core_per_host, lr_layer_decay_rate, use_tpu, learning_rate, train_steps,
                min_lr_ratio, clip, **kwargs):
        self.decay_method = decay_method
        self.warmup_steps = warmup_steps
        self.weight_decay = weight_decay
        self.adam_epsilon = adam_epsilon
        self.num_core_per_host = num_core_per_host
        self.lr_layer_decay_rate = lr_layer_decay_rate
        self.use_tpu = use_tpu
        self.learning_rate = learning_rate
        self.train_steps = train_steps
        self.min_lr_ratio = min_lr_ratio
        self.clip = clip
        
training_parameters = Parameter(**training_parameters)

In [10]:
class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.float32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        xlnet_model = xlnet.XLNetModel(
            xlnet_config=xlnet_config,
            run_config=xlnet_parameters,
            input_ids=tf.transpose(self.X, [1, 0]),
            seg_ids=tf.transpose(self.segment_ids, [1, 0]),
            input_mask=tf.transpose(self.input_masks, [1, 0]))
        
        summary = xlnet_model.get_pooled_out("last", True)
        print(summary)
        
        self.logits = tf.layers.dense(summary, dimension_output)
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        self.learning_rate = learning_rate
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        
#         self.optimizer, self.learning_rate, _ = model_utils.get_train_op(training_parameters, self.cost)
        
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [11]:
dimension_output = 2

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())

W0716 17:06:45.170339 140390586017536 deprecation_wrapper.py:119] From /home/jupyter/xlnet/xlnet.py:220: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0716 17:06:45.171341 140390586017536 deprecation_wrapper.py:119] From /home/jupyter/xlnet/xlnet.py:220: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0716 17:06:45.172147 140390586017536 deprecation_wrapper.py:119] From /home/jupyter/xlnet/modeling.py:451: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

W0716 17:06:45.172909 140390586017536 deprecation_wrapper.py:119] From /home/jupyter/xlnet/modeling.py:458: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0716 17:06:45.243463 140390586017536 deprecation.py:323] From /home/jupyter/xlnet/modeling.py:533: dropout (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:

Tensor("model_1/sequnece_summary/summary/Tanh:0", shape=(?, 256), dtype=float32)


In [12]:
import collections
import re

def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if name not in name_to_variable:
            continue
        assignment_map[name] = name_to_variable[name]
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ':0'] = 1

    return (assignment_map, initialized_variable_names)

In [13]:
tvars = tf.trainable_variables()
tvars

[<tf.Variable 'model/transformer/r_w_bias:0' shape=(6, 16, 64) dtype=float32_ref>,
 <tf.Variable 'model/transformer/r_r_bias:0' shape=(6, 16, 64) dtype=float32_ref>,
 <tf.Variable 'model/transformer/word_embedding/lookup_table:0' shape=(32000, 256) dtype=float32_ref>,
 <tf.Variable 'model/transformer/r_s_bias:0' shape=(6, 16, 64) dtype=float32_ref>,
 <tf.Variable 'model/transformer/seg_embed:0' shape=(6, 2, 16, 64) dtype=float32_ref>,
 <tf.Variable 'model/transformer/layer_0/rel_attn/q/kernel:0' shape=(256, 16, 64) dtype=float32_ref>,
 <tf.Variable 'model/transformer/layer_0/rel_attn/k/kernel:0' shape=(256, 16, 64) dtype=float32_ref>,
 <tf.Variable 'model/transformer/layer_0/rel_attn/v/kernel:0' shape=(256, 16, 64) dtype=float32_ref>,
 <tf.Variable 'model/transformer/layer_0/rel_attn/r/kernel:0' shape=(256, 16, 64) dtype=float32_ref>,
 <tf.Variable 'model/transformer/layer_0/rel_attn/o/kernel:0' shape=(256, 16, 64) dtype=float32_ref>,
 <tf.Variable 'model/transformer/layer_0/rel_attn/L

In [14]:
initialized_variable_names = {}
checkpoint = 'output-model4/model.ckpt'
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                checkpoint)

In [15]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, checkpoint)

W0716 17:06:55.291638 140390586017536 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [16]:
from sklearn.model_selection import train_test_split

train_input_ids, test_input_ids, train_input_masks, test_input_masks, train_segment_ids, test_segment_ids, train_Y, test_Y = train_test_split(
    input_ids, input_masks, segment_ids, labels, test_size = 0.2
)

In [17]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_masks = train_input_masks[i: index]
        batch_segment = train_segment_ids[i: index]
        batch_y = train_Y[i: index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_masks = test_input_masks[i: index]
        batch_segment = test_segment_ids[i: index]
        batch_y = test_Y[i: index]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_input_ids) / batch_size
    train_acc /= len(train_input_ids) / batch_size
    test_loss /= len(test_input_ids) / batch_size
    test_acc /= len(test_input_ids) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 3/797 [00:00<00:37, 21.45it/s, accuracy=0.8, cost=0.443]

epoch: 0, pass acc: 0.000000, current acc: 0.895969
time taken: 44.12548756599426
epoch: 0, training loss: 0.345737, training acc: 0.846781, valid loss: 0.261241, valid acc: 0.895969



train minibatch loop:   0%|          | 3/797 [00:00<00:37, 21.16it/s, accuracy=0.8, cost=0.27]

epoch: 1, pass acc: 0.895969, current acc: 0.906004
time taken: 41.38458967208862
epoch: 1, training loss: 0.195034, training acc: 0.927092, valid loss: 0.242538, valid acc: 0.906004



train minibatch loop:   0%|          | 3/797 [00:00<00:37, 21.38it/s, accuracy=1, cost=0.103] 

...

time taken: 42.06927180290222
epoch: 9, training loss: 0.001641, training acc: 0.999749, valid loss: 0.499257, valid acc: 0.919050



train minibatch loop:   0%|          | 3/797 [00:00<00:37, 21.17it/s, accuracy=1, cost=1.3e-5] 

time taken: 41.492979526519775
epoch: 10, training loss: 0.000770, training acc: 1.000000, valid loss: 0.560301, valid acc: 0.916541



test minibatch loop: 100%|██████████| 200/200 [00:03<00:00, 52.92it/s, accuracy=0.667, cost=3]   

time taken: 40.976240158081055
epoch: 11, training loss: 0.016038, training acc: 0.994353, valid loss: 0.477517, valid acc: 0.913531

break epoch:12

